In [ ]:
import json
import time
import os
import sys
import boto3
from botocore.exceptions import ClientError
from boto3.dynamodb.conditions import Key, Attr


In [ ]:
# ----- CONFIG -----
AWS_REGION = os.environ.get("AWS_REGION", "us-east-1")
TABLE_NAME = "Pedido-AndreySilva"  # substitua 'seunome' pelo seu nome, ex: Pedido-andrey
LSI_NAME = "LSI-PedidoAndreySilvaStatus"
GSI_NAME = "GLI-Status-ValorTotal"
# Arquivo opcional com itens para batch insert (Python list of dicts)
BULK_JSON_FILE = "pedidos_import.json"  # fará import via batch_writer
# ------------------

dynamodb = boto3.resource("dynamodb", region_name=AWS_REGION)
client = boto3.client("dynamodb", region_name=AWS_REGION)

def create_table_with_lsi():
    """
    Cria a tabela com chave de partição UserID (string) e chave de ordenação OrderDate (string).
    Adiciona um LSI com chave de ordenação Status (string) e projeção ALL.
    Provisioned throughput 5 read / 5 write (conforme doc).
    """
    try:
        print(f"Creating table {TABLE_NAME} with LSI {LSI_NAME} ...")
        table = client.create_table(
            TableName=TABLE_NAME,
            KeySchema=[
                {"AttributeName": "UserID", "KeyType": "HASH"},     # Partition key
                {"AttributeName": "OrderDate", "KeyType": "RANGE"}  # Sort key
            ],
            AttributeDefinitions=[
                {"AttributeName": "UserID", "AttributeType": "S"},
                {"AttributeName": "OrderDate", "AttributeType": "S"},
                {"AttributeName": "Status", "AttributeType": "S"},  # required for LSI sort key
            ],
            LocalSecondaryIndexes=[
                {
                    "IndexName": LSI_NAME,
                    "KeySchema": [
                        {"AttributeName": "UserID", "KeyType": "HASH"},
                        {"AttributeName": "Status", "KeyType": "RANGE"}
                    ],
                    "Projection": {"ProjectionType": "ALL"}
                }
            ],
            BillingMode="PROVISIONED",
            ProvisionedThroughput={"ReadCapacityUnits": 5, "WriteCapacityUnits": 5}
        )
        print("CreateTable called. Waiting for ACTIVE status...")
        waiter = client.get_waiter('table_exists')
        waiter.wait(TableName=TABLE_NAME)
        # Wait until table is ACTIVE
        while True:
            desc = client.describe_table(TableName=TABLE_NAME)
            status = desc['Table']['TableStatus']
            print("Table status:", status)
            if status == 'ACTIVE':
                break
            time.sleep(3)
        print("Table created and ACTIVE.")
        return dynamodb.Table(TABLE_NAME)
    except ClientError as e:
        print("Error creating table:", e.response.get('Error', e))
        raise

def put_sample_item():
    """Insere o item manual do passo 3 (U000 / 2025-05-16)"""
    print("Inserting sample item...")
    table = dynamodb.Table(TABLE_NAME)
    item = {
        "UserID": "U000",
        "OrderDate": "2025-05-16",
        "Endereco": {
            "bairro": "Santa Maria",
            "cidade": "Recife",
            "estado": "PE",
            "numero": "S/D",
            "rua": "Cavalcante Novaes 2"
        },
        "tags": ["Entrega zona rural", "Equipe Rocket", "Recife"],
        "IDPedido": "P1000",
        "Status": "Pendente",
        "ValorTotal": 900
    }
    table.put_item(Item=item)
    print("Sample item inserted.")

def import_json_bulk(filepath=BULK_JSON_FILE):
    """
    Importa items de um arquivo JSON no formato de lista de objetos.
    Exemplo esperado: [ { "UserID": "...", "OrderDate": "...", ...}, {...} ]
    Usa Table.batch_writer() para lidar com lots e throttling.
    """
    if not os.path.isfile(filepath):
        print(f"No bulk file '{filepath}' found — pulando import.")
        return

    print(f"Importing items from {filepath} ...")
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    if not isinstance(data, list):
        print("Arquivo JSON esperado: lista de objetos. Abortando import.")
        return

    table = dynamodb.Table(TABLE_NAME)
    with table.batch_writer() as batch:
        count = 0
        for obj in data:
            # itens já no formato python dict (números como numbers, strings como strings)
            batch.put_item(Item=obj)
            count += 1
    print(f"Imported {count} items from {filepath}.")

def query_by_user(user_id="U001"):
    """Consulta por chave de partição (Query). Retorna todos os pedidos do user_id."""
    print(f"Querying by UserID={user_id} ...")
    table = dynamodb.Table(TABLE_NAME)
    resp = table.query(
        KeyConditionExpression=Key('UserID').eq(user_id)
    )
    items = resp.get('Items', [])
    print(f"Found {len(items)} items for {user_id}.")
    for it in items:
        print(it)
    return items

def query_lsi_status(user_id="U001", status_value="Entregue"):
    """
    Consulta usando o LSI (index local: partition Key + status sort key).
    Requer IndexName set to LSI_NAME.
    """
    print(f"Querying LSI index {LSI_NAME} for UserID={user_id} AND Status={status_value} ...")
    table = dynamodb.Table(TABLE_NAME)
    resp = table.query(
        IndexName=LSI_NAME,
        KeyConditionExpression=Key('UserID').eq(user_id) & Key('Status').eq(status_value)
    )
    items = resp.get('Items', [])
    print(f"LSI query returned {len(items)} items.")
    for it in items:
        print(it)
    return items

def create_gsi_status_valortotal():
    """
    Cria um Global Secondary Index com partition key Status (S) e sort key ValorTotal (N).
    Nome: GSI_NAME
    OBS: precisa declarar AttributeDefinitions correspondentes.
    """
    print(f"Creating GSI {GSI_NAME} ... this may take some minutes to become ACTIVE.")
    try:
        resp = client.update_table(
            TableName=TABLE_NAME,
            AttributeDefinitions=[
                {"AttributeName": "Status", "AttributeType": "S"},
                {"AttributeName": "ValorTotal", "AttributeType": "N"}
            ],
            GlobalSecondaryIndexUpdates=[
                {
                    "Create": {
                        "IndexName": GSI_NAME,
                        "KeySchema": [
                            {"AttributeName": "Status", "KeyType": "HASH"},
                            {"AttributeName": "ValorTotal", "KeyType": "RANGE"}
                        ],
                        "Projection": {"ProjectionType": "ALL"},
                        "ProvisionedThroughput": {"ReadCapacityUnits": 5, "WriteCapacityUnits": 5}
                    }
                }
            ]
        )
    except ClientError as e:
        print("Error creating GSI:", e.response.get('Error', e))
        raise

    # Poll until index is ACTIVE
    print("Waiting for GSI to become ACTIVE ...")
    while True:
        desc = client.describe_table(TableName=TABLE_NAME)
        gsis = desc['Table'].get('GlobalSecondaryIndexes') or []
        found = next((g for g in gsis if g['IndexName'] == GSI_NAME), None)
        if found:
            status = found.get('IndexStatus')
            print(f"GSI status: {status}")
            if status == 'ACTIVE':
                print("GSI is ACTIVE.")
                break
        else:
            print("GSI not found yet, waiting...")
        time.sleep(5)
    return True

def query_gsi_status_min_val(status_value="Entregue", min_val=60):
    """
    Consulta no GSI por Status = status_value e ValorTotal >= min_val.
    Usamos KeyConditionExpression (status equality + valor range >=).
    """
    print(f"Querying GSI {GSI_NAME} for Status={status_value} and ValorTotal >= {min_val} ...")
    table = dynamodb.Table(TABLE_NAME)
    # KeyConditionExpression supports comparison on sort key
    resp = table.query(
        IndexName=GSI_NAME,
        KeyConditionExpression=Key('Status').eq(status_value) & Key('ValorTotal').gte(min_val)
    )
    items = resp.get('Items', [])
    print(f"GSI query returned {len(items)} items.")
    for it in items:
        print(it)
    return items

def delete_gsi():
    """Remove o GSI criado."""
    print(f"Deleting GSI {GSI_NAME} ...")
    try:
        client.update_table(
            TableName=TABLE_NAME,
            GlobalSecondaryIndexUpdates=[
                {"Delete": {"IndexName": GSI_NAME}}
            ]
        )
    except ClientError as e:
        print("Error deleting GSI:", e.response.get('Error', e))
        raise

    # Wait until GSI disappears from describe_table
    print("Waiting for GSI deletion to finish ...")
    while True:
        desc = client.describe_table(TableName=TABLE_NAME)
        gsis = desc['Table'].get('GlobalSecondaryIndexes') or []
        found = next((g for g in gsis if g['IndexName'] == GSI_NAME), None)
        if not found:
            print("GSI deleted.")
            break
        print("Still deleting ...")
        time.sleep(5)

def delete_table():
    """Exclui a tabela DynamoDB."""
    print(f"Deleting table {TABLE_NAME} ...")
    try:
        client.delete_table(TableName=TABLE_NAME)
    except ClientError as e:
        print("Error deleting table:", e.response.get('Error', e))
        raise

    # Wait until table no longer exists
    print("Waiting for table deletion ...")
    while True:
        try:
            client.describe_table(TableName=TABLE_NAME)
            print("Table still exists ... waiting.")
            time.sleep(5)
        except client.exceptions.ResourceNotFoundException:
            print("Table successfully deleted.")
            break

def main():
    print("DynamoDB lab script starting...")
    # 1) Create table with LSI
    try:
        table = create_table_with_lsi()
    except Exception as e:
        print("Could not create table:", e)
        sys.exit(1)

    # 2) Insert sample item
    put_sample_item()

    # 3) Bulk import if file exists
    import_json_bulk()

    # 4) Query by user (U001 example)
    # Note: if your data uses U000 only, adjust user_id below to test accordingly.
    print("\n--- Query by partition key (example U001) ---")
    query_by_user(user_id="U001")

    # 5) Query with LSI (UserID + Status)
    print("\n--- Query with LSI (UserID=U001, Status=Entregue) ---")
    query_lsi_status(user_id="U001", status_value="Entregue")

    # 6) Create GSI
    print("\n--- Creating GSI (Status / ValorTotal) ---")
    create_gsi_status_valortotal()

    # 7) Query using GSI (Status = 'Entregue', ValorTotal >= 60)
    print("\n--- Query using GSI (Status='Entregue' and ValorTotal >= 60) ---")
    query_gsi_status_min_val(status_value="Entregue", min_val=60)

    # 8) Cleanup: delete GSI and table
    print("\n--- Cleaning up: deleting GSI and table ---")
    delete_gsi()
    delete_table()
    print("Lab finished successfully.")

if __name__ == "__main__":
    main()
